In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import partial
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/instacart-market-basket-analysis/products.csv
/kaggle/input/instacart-market-basket-analysis/order_products__train.csv
/kaggle/input/instacart-market-basket-analysis/orders.csv
/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv
/kaggle/input/instacart-market-basket-analysis/aisles.csv
/kaggle/input/instacart-market-basket-analysis/departments.csv


In [2]:
aisles = pd.read_csv('../input/instacart-market-basket-analysis/aisles.csv')
depts = pd.read_csv('../input/instacart-market-basket-analysis/departments.csv')
orders_prior = pd.read_csv('../input/instacart-market-basket-analysis/order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})
orders_train = pd.read_csv('../input/instacart-market-basket-analysis/order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})
orders_full = pd.read_csv('../input/instacart-market-basket-analysis/orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float16})
products = pd.read_csv('../input/instacart-market-basket-analysis/products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

In [3]:
orders_train['reordered'].describe()

count    1.384617e+06
mean     5.985944e-01
std      4.901829e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: reordered, dtype: float64

In [4]:
products = products.set_index('product_id')
products.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [5]:
#Add a field to calculate the sum of times an item was reordered
products['rsum']=orders_train.groupby('product_id')['reordered'].sum()
#Add a field to calculate the total times the item could have been reordered
products['rtotal']=orders_train.groupby('product_id')['reordered'].count()
#Add a field to calculate the probability that the item was reordered
products['prob']=products['rsum']/products['rtotal']
products = products.reset_index()
# groupby sorts by grouping key so we can directly assign to products df without risking 
# getting entries mismatched because the products df is ordered by product id


In [6]:
products.head()

,product_id,product_name,aisle_id,department_id,rsum,rtotal,prob
0,1,Chocolate Sandwich Cookies,61,19,49.0,76.0,0.644737
1,2,All-Seasons Salt,104,13,1.0,4.0,0.250000
2,3,Robust Golden Unsweetened Oolong Tea,94,7,6.0,6.0,1.000000
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,14.0,22.0,0.636364
4,5,Green Chile Anytime Sauce,5,13,1.0,1.0,1.000000


In [7]:
# Merge all of the details into a goods page
# Merges on common columns if 'on' argument not specified
goods = pd.merge(left=pd.merge(left=products, right=depts, how='left'), right=aisles, how='left')

In [8]:
goods.head()

,product_id,product_name,aisle_id,department_id,rsum,rtotal,prob,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,49.0,76.0,0.644737,snacks,cookies cakes
1,2,All-Seasons Salt,104,13,1.0,4.0,0.250000,pantry,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,6.0,6.0,1.000000,beverages,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,14.0,22.0,0.636364,frozen,frozen meals
4,5,Green Chile Anytime Sauce,5,13,1.0,1.0,1.000000,pantry,marinades meat preparation


In [9]:
# merge train and prior together iteratively, to fit into 8GB kernel RAM
order_details = pd.merge(
                left=orders_train,
                 right=orders_full, 
                 how='left', 
                 on='order_id'
        ).apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))

In [10]:
# add order hierarchy
order_details = pd.merge(
                left=order_details,
                right=goods[['product_id', 
                             'aisle_id', 
                             'department_id',
                             'prob']].apply(partial(pd.to_numeric,
                                                             errors='ignore', 
                                                             downcast='integer')),
                how='left',
                on='product_id'
)

In [11]:
print(order_details.shape, orders_train.shape)

# delete (redundant now) dataframes
del orders_train

order_details.head()

(1384617, 13) (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,prob
0,1,49302,1,1,112108,train,4,4,10,9,120,16,0.875000
1,1,11109,2,1,112108,train,4,4,10,9,108,16,0.743056
2,1,10246,3,0,112108,train,4,4,10,9,83,4,0.535782
3,1,49683,4,0,112108,train,4,4,10,9,83,4,0.708247
4,1,43633,5,1,112108,train,4,4,10,9,95,15,0.416667


In [12]:
# split df indexes into parts
indexes = np.linspace(0, len(orders_prior), num=10, dtype=np.int32)# initialize it with train dataset
indexes

array([       0,  3603832,  7207664, 10811496, 14415328, 18019160,
       21622992, 25226824, 28830656, 32434489], dtype=int32)

In [13]:
for i in range(len(indexes)-1):
    order_details = pd.concat(
        [   
            order_details,
            pd.merge(left=pd.merge(
                            left=orders_prior.loc[indexes[i]:indexes[i+1], :],
                            right=goods[['product_id', 
                                         'aisle_id', 
                                         'department_id' ]].apply(partial(pd.to_numeric, 
                                                                          errors='ignore', 
                                                                          downcast='integer')),
                            how='left',
                            on='product_id'
                            ),
                     right=orders_full, 
                     how='left', 
                     on='order_id'
                ) 
        ]
    )

In [14]:
order_details

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,prob
0,1,49302,1,1,112108,train,4,4,10,9.0,120,16,0.875000
1,1,11109,2,1,112108,train,4,4,10,9.0,108,16,0.743056
2,1,10246,3,0,112108,train,4,4,10,9.0,83,4,0.535782
3,1,49683,4,0,112108,train,4,4,10,9.0,83,4,0.708247
4,1,43633,5,1,112108,train,4,4,10,9.0,95,15,0.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603828,3421083,39678,6,1,25247,prior,24,2,6,21.0,74,17,NaN
3603829,3421083,11352,7,0,25247,prior,24,2,6,21.0,78,19,NaN
3603830,3421083,4600,8,0,25247,prior,24,2,6,21.0,52,1,NaN
3603831,3421083,24852,9,1,25247,prior,24,2,6,21.0,24,4,NaN


In [15]:
order_details['eval_set'] = order_details['eval_set'].astype('category')

In [16]:
print('Datafame length: {}'.format(order_details.shape[0]))
print('Memory consumption: {:.2f} Mb'.format(sum(order_details.memory_usage(index=True, 
                                                                         deep=True) / 2**20)))
# check dtypes to see if we use memory effectively
print(order_details.dtypes)

Datafame length: 33819114
Memory consumption: 1225.59 Mb
order_id                     int32
product_id                  uint16
add_to_cart_order            int16
reordered                     int8
user_id                      int32
eval_set                  category
order_number                 int16
order_dow                     int8
order_hour_of_day             int8
days_since_prior_order     float16
aisle_id                     uint8
department_id                 int8
prob                       float64
dtype: object


In [17]:
# make sure we didn't forget to retain test dataset :D
test_orders = orders_full[orders_full.eval_set == 'test']

# delete (redundant now) dataframes
del orders_prior, orders_full

In [18]:
test_history = order_details[(order_details.user_id.isin(test_orders.user_id))]
last_orders = test_history.groupby('user_id')['order_number'].max()

In [19]:
def get_last_orders_reordered():
    t = pd.merge(
            left=pd.merge(
                    left=last_orders.reset_index(),
                    right=test_history[test_history.reordered == 1],
                    how='left',
                    on=['user_id', 'order_number']
                )[['user_id', 'product_id']],
            right=test_orders[['user_id', 'order_id']],
            how='left',
            on='user_id'
        ).fillna(-1).groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(int(e)) for e in set(x)]) 
                                                  ).reset_index().replace(to_replace='-1', 
                                                                          value='None')
    t.columns = ['order_id', 'products']
    return t

In [20]:
get_last_orders_reordered().to_csv('last_order_reordered_only.csv', 
                         encoding='utf-8', 
                         index=False)